In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(42)
np.random.seed(42)

2026-01-06 08:53:36.565340: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-06 08:53:36.572264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767669816.580659   45067 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767669816.583081   45067 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767669816.589408   45067 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
subset = 10000
x_train_small = x_train[:subset]
y_train_small = y_train[:subset]
input_shape = x_train_small.shape[1:]
num_classes = 10

In [4]:
input_shape

(32, 32, 3)

In [5]:
strong_aug = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.GaussianNoise(0.05),
])

I0000 00:00:1767670361.405160   45067 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1134 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
def identity_aug(x):
    return x

In [10]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 256
temperature = 0.1
epochs = 6

def make_simclr_ds(images, aug_fn):
    ds = tf.data.Dataset.from_tensor_slices(images)
    ds = ds.shuffle(10000)
    ds = ds.map(lambda x :( aug_fn(x, training=True), aug_fn(x, training=True)), num_parallel_calls=AUTOTUNE)
    return ds

In [11]:
ds_aug = make_simclr_ds(x_train_small, strong_aug)
ds_plain = make_simclr_ds(x_train_small, lambda x : identity_aug(x))

TypeError: in user code:

    File "/tmp/ipykernel_45067/3130662467.py", line 9, in None  *
        lambda x :( aug_fn(x, training=True), aug_fn(x, training=True))

    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() got an unexpected keyword argument 'training'
